# Pre-process EBSD patterns from an Al-Mn alloy

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

See the relevant package documentation for more details on the packages used here:
* diffsims: https://diffsims.readthedocs.io/en/latest/
* kikuchipy: https://kikuchipy.org/en/stable/
* hyperspy: https://hyperspy.org/hyperspy-doc/current/
* pyebsdindex: https://pyebsdindex.readthedocs.io/en/latest/
* orix: https://orix.readthedocs.io/en/stable/

Import required packages and print their versions

In [21]:
# Replace "inline" with "qt5" from the pyqt package for interactive plotting
%matplotlib qt5

from datetime import date
import importlib_metadata
import os
from time import time

import dask
from dask.diagnostics import ProgressBar
from diffpy.structure import Lattice, Structure
from diffsims.crystallography import ReciprocalLatticeVector
import hyperspy.api as hs
import kikuchipy as kp
import matplotlib.pyplot as plt
import numpy as np
from pyebsdindex import ebsd_index, pcopt
from orix import io, plot, sampling
from orix.crystal_map import CrystalMap, PhaseList
from orix.quaternion import Rotation


# Directories
sample = "325c"
dset_no = 3
dir_mp = "/home/hakon/kode/emsoft/emdata/crystal_data"
dir_data = os.path.join("/home/hakon/phd/data/p/prover", sample, str(dset_no))
dir_nordif = os.path.join(dir_data, "nordif")
dir_kp = os.path.join(dir_data, "kp")

# Data set specific parameters
cal_pats_al = {
    "0s": {
        1: [0, 1, 2, 3, 4],
        2: [0, 1, 2, 3, 4],
        3: [0, 1, 2, 3, 4]
    },
    "325c": {
        1: [0, 10, 11, 12, 13, 14],
        2: [6, 7, 8, 9],
        3: [0, 1, 2, 3, 4, 5],
    }
}
pc0 = {
    "0s": {
        1: (0.513, 0.120, 0.550),
        2: (0.506, 0.180, 0.580),
        3: (0.649, 0.135, 0.707),
    },
    "325c": {
        1: (0.5, 0.2, 0.5),
        2: (0.49689, 0.2083, 0.5560),
        3: (0.4970, 0.1348, 0.5527),
    }
}

# Matplotlib
plt.rcParams.update({"figure.figsize": (5, 5), "font.size": 12})
savefig_kw = dict(bbox_inches="tight", pad_inches=0, dpi=150)

print("Run date: ", date.today())
print("\nSoftware versions\n------------------")
for pkg in [
    "dask",
    "diffpy.structure",
    "diffsims",
    "hyperspy",
    "kikuchipy",
    "matplotlib",
    "numpy",
    "pyebsdindex",
    "orix",
]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

Run date:  2022-06-18

Software versions
------------------
dask : 2022.6.0
diffpy.structure : 3.0.1
diffsims : 0.5.0
hyperspy : 1.7.0
kikuchipy : 0.6.1
matplotlib : 3.5.2
numpy : 1.22.4
pyebsdindex : 0.1rc2
orix : 0.9.0.post0


# Pre-correction maps

Load data lazily (not into RAM)

In [3]:
s = kp.load(os.path.join(dir_kp, "patterns_dewrap.h5"), lazy=True)
s.static_background = plt.imread(os.path.join(dir_nordif, "Background acquisition pattern.bmp"))

sig_shape = s.axes_manager.signal_shape[::-1]
s

<LazyEBSD, title: patterns_dewrap Scan..., dimensions: (1028, 1026|60, 60)>

Mean intensity map

In [4]:
s_mean = s.mean(axis=s.axes_manager.signal_axes)
s_mean.compute()

[########################################] | 100% Completed |  7.1s


Save unprocessed map and contrast stretched map

In [5]:
# Unprocessed map
map_mean1 = s_mean.data
plt.imsave(os.path.join(dir_kp, "maps_mean.png"), map_mean1, cmap="gray")

# Contrast stretched map
percentiles = np.percentile(map_mean1, q=(1, 99))
map_mean2 = kp.pattern.rescale_intensity(map_mean1, in_range=percentiles)
plt.imsave(os.path.join(dir_kp, "maps_mean_q1_q99.png"), map_mean2, cmap="gray")

Generate an RGB virtual backscatter electron (VBSE) image. First, set up the generator

In [6]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)

Plot all grid tiles and hightlight the RGB tiles

In [7]:
vbse_gen.grid_shape = (5, 5)
red = (2, 1)
green = (2, 2)
blue = (2, 3)
vbse_grid_plot = vbse_gen.plot_grid(
    rgb_channels=[red, green, blue], pattern_idx=(0, 0)
)
vbse_grid_plot._plot.signal_plot.figure.savefig(
    os.path.join(dir_kp, "vbse5x5_grid_plot.png"), **savefig_kw
)

plt.close("all")

Generate the VBSE RGB image

In [8]:
vbse_rgb = vbse_gen.get_rgb_image(r=red, g=green, b=blue)
vbse_rgb.save(os.path.join(dir_kp, "vbse5x5_rgb.png"))

2022-06-18 14:58:21,973 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-06-18 14:58:23,253 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.
2022-06-18 14:58:24,365 - hyperspy.io - WARNING - `signal_type='VirtualBSEImage'` not understood. See `hs.print_known_signal_types()` for a list of installed signal types or https://github.com/hyperspy/hyperspy-extensions-list for the list of all hyperspy extensions providing signals.


Background correction:
1. Remove static background
2. Remove dynamic background
3. Average patterns with their eight nearest neighbour using a Gaussian kernel with $\sigma$ = 1

In [9]:
s.remove_static_background()

In [10]:
s.remove_dynamic_background()

In [11]:
w = kp.filters.Window(window="gaussian", std=1)

In [12]:
s.average_neighbour_patterns(window=w)

Write processed patterns to file

In [13]:
with ProgressBar():
    s.save(os.path.join(dir_kp, "pattern_sda.h5"))

[########################################] | 100% Completed |  3min 52.3s


## Pre-indexing maps

Generate image quality $\mathbf{Q}$ and average dot product maps

In [14]:
s = kp.load(os.path.join(dir_kp, "pattern_sda.h5"), lazy=True)
s

<LazyEBSD, title: pattern_sda Scan 1, dimensions: (1028, 1026|60, 60)>

In [15]:
iq_dask = s.get_image_quality()

with ProgressBar():
    iq = iq_dask.compute()

plt.imsave(os.path.join(dir_kp, "maps_iq.png"), arr=iq, cmap="gray")

[########################################] | 100% Completed |  1min  8.3s


In [16]:
adp_dask = s.get_average_neighbour_dot_product_map()

with ProgressBar():
    adp = adp_dask.compute()

plt.imsave(os.path.join(dir_kp, "maps_adp.png"), arr=adp, cmap="gray")

[########################################] | 100% Completed |  9min 37.7s


Plot maps

In [17]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(iq, cmap="gray")
ax[1].imshow(adp, cmap="gray")
ax[0].axis("off")
ax[1].axis("off")
fig.tight_layout()

## Projection center from PyEBSDIndex

Load calibration patterns

In [18]:
s_cal0 = kp.load(os.path.join(dir_nordif, "Setting.txt"))
sig_shape_cal = s_cal0.axes_manager.signal_shape[::-1]
s_cal0

<EBSD, title: Calibration patterns, dimensions: (6|480, 480)>

Increase the signal-to-noise ratio

In [19]:
s_cal0.remove_static_background()
s_cal0.remove_dynamic_background()

Removing the static background:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
Removing the dynamic background:
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [20]:
s_cal0.plot()

Extract Al calibration patterns

In [22]:
s_cal = kp.signals.EBSD(s_cal0.data[cal_pats_al[sample][dset_no]])
s_cal.axes_manager[0].name = "x"
nav_size = s_cal.axes_manager.navigation_size

Extract relevant metadata

In [23]:
md_sem = s_cal0.metadata.Acquisition_instrument.SEM
md_ebsd = md_sem.Detector.EBSD
sample_tilt = md_ebsd.sample_tilt  # Degrees
camera_tilt = md_ebsd.azimuth_angle  # Degrees
energy = md_sem.beam_energy  # kV

Generate an indexer instance with PyEBSDIndex for easy storage of relevant
parameters used in projection center (PC) optimization

In [24]:
indexer = ebsd_index.EBSDIndexer(
    phaselist=["FCC"],
    vendor="BRUKER",
    PC=None,
    sampleTilt=sample_tilt,
    camElev=camera_tilt,
    patDim=sig_shape_cal,
)

Define an EBSD detector without a specific PC set

In [25]:
detector = kp.detectors.EBSDDetector(
    shape=sig_shape_cal,
    sample_tilt=sample_tilt,
    tilt=camera_tilt,
)

Load Al master pattern to use in PC refinement and to extract the Al crystal structure

In [26]:
mp = kp.load(
    os.path.join(dir_mp, "al", "al_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy,
    hemisphere="upper",
)
mp.phase.name = "al"

Inspect geometrical simulations

In [27]:
ref = ReciprocalLatticeVector(
    phase=mp.phase, hkl=((1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1))
)
ref = ref.symmetrise().unique()
ref.print_table()

 h k l      d     |F|_hkl   |F|^2   |F|^2_rel   Mult 
 3 1 1    0.122     nan      nan       nan       24  
 1 1 1    0.233     nan      nan       nan       8   
 2 2 0    0.143     nan      nan       nan       12  
 2 0 0    0.202     nan      nan       nan       6   


In [28]:
simulator = kp.simulations.KikuchiPatternSimulator(ref)

In [29]:
simulator.plot()

Find PC from single pattern using an initial guess, as a test

In [54]:
pattern = s_cal.inav[0].data

pc0_i = (0.4842, 0.1320, 0.5466)
pc = pcopt.optimize(pattern, indexer, PC0=pc0_i)
print("PC: ", pc)

data = indexer.index_pats(pattern, PC=pc)[0]
rot = Rotation(data["quat"][-1]) * Rotation.from_axes_angles((0, 0, -1), np.pi / 2)
print("Fit: ", data["fit"][-1][0])

detector.pc = pc
geosim = simulator.on_detector(detector, rot)
geosim.plot(pattern=pattern, zone_axes_labels=False, zone_axes=False)

PC:  [0.48423261 0.13196559 0.54655104]
Fit:  0.81284523
Finding bands that are in some pattern:
[########################################] | 100% Completed |  0.1s
Finding zone axes that are in some pattern:
[########################################] | 100% Completed |  0.1s
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed |  0.1s


Find PC from all patterns

In [55]:
pcs = np.zeros((nav_size, 3))
for i in range(nav_size):
    pcs[i] = pcopt.optimize(s_cal.inav[i].data, indexer, PC0=pc)
print(pcs)

pc = pcs.mean(axis=0)
print(pc)

[[0.48423261 0.13196559 0.54655104]
 [0.50740462 0.13010225 0.55332459]
 [0.4900785  0.1350787  0.55056136]
 [0.4885871  0.1410818  0.56044398]
 [0.51510815 0.14166006 0.54616885]
 [0.49661281 0.12874948 0.55910983]]
[0.49700397 0.13477298 0.55269327]


Index calibration patterns to check PCs

In [56]:
data = indexer.index_pats(patsin=s_cal.data, PC=pcs)[0]
rot = Rotation(data["quat"][-1]) * Rotation.from_axes_angles((0, 0, -1), np.pi / 2)
print(data["fit"][-1])

[0.81284523 0.6172587  0.70641977 0.7907745  0.6424133  0.31427395]


Update detector instance

In [57]:
detector.pc = pcs

In [58]:
geosim = simulator.on_detector(detector, rot)

Finding bands that are in some pattern:
[########################################] | 100% Completed |  0.1s
Finding zone axes that are in some pattern:
[########################################] | 100% Completed |  0.1s
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed |  0.1s


In [59]:
s_cal.add_marker(geosim.as_markers())

Refine results from PyEBSDIndex

In [60]:
xmap = CrystalMap(rotations=rot, phase_list=PhaseList(mp.phase))

# First refine orientations, then projection centers
ref_kwargs = dict(detector=detector, master_pattern=mp, energy=energy)
xmap_refined = s_cal.refine_orientation(xmap=xmap, **ref_kwargs)
_, detector_ref = s_cal.refine_projection_center(xmap=xmap_refined, **ref_kwargs)

Refinement information:
	Local optimization method: Nelder-Mead (minimize)
	Keyword arguments passed to method: {'method': 'Nelder-Mead'}
Refining 6 orientation(s):
[########################################] | 100% Completed | 10.0s
Refinement speed: 0 patterns/s
Refinement information:
	Local optimization method: Nelder-Mead (minimize)
	Keyword arguments passed to method: {'method': 'Nelder-Mead'}
Refining 6 projection center(s):
[########################################] | 100% Completed |  7.6s
Refinement speed: 0 patterns/s


Check geometrical simulations of refined orientations and PCs

In [61]:
geosim_ref = simulator.on_detector(detector_ref, xmap_refined.rotations)

Finding bands that are in some pattern:
[########################################] | 100% Completed |  0.1s
Finding zone axes that are in some pattern:
[########################################] | 100% Completed |  0.1s
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed |  0.1s


In [62]:
s_cal.add_marker(geosim_ref.as_markers())

In [63]:
np.savetxt(
    os.path.join(dir_kp, "cal_pcs.txt"),
    np.column_stack((cal_pats_al[sample][dset_no], detector_ref.pc)),
    fmt="%i %.12f %.12f %.12f",
    header="Cal. pattern, PC (x, y, z) in Bruker's convention"
)

## Dictionary indexing

Done in a separate notebook using the above obtained PC.

## Refinement

Done in a separate notebook using the DI results.